In [1]:
## Prepare the expression summary stats - paired (that take both genes into account)

# Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"


Loading required package: limma

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.8-41. For overview type 'help("mgcv-package")'.

Loading required package: genefilter

Loading required package: BiocParallel

corrplot 0.92 loaded

Loading required package: viridisLite


Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose



Attaching package: ‘MASS’


The following object is masked from ‘package:genefilter’:

    area



Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘SeuratObject’


The following object is masked from 

In [3]:
library(stringi)

# Parameters

In [4]:
### Define dataset and cell-type for which to prepare

In [5]:
dataset = 'oneK1k'

In [6]:
# Path to the expression data

In [7]:
expression_data = '/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/1K1K/expression_data/'

In [ ]:
### Path to save calculated summaries to

In [13]:
results_path_analysis =  paste0('/lustre/groups/epigenereg01/workspace/projects/grn_dev_groningen/coeqtl_mapping/co_qtls_sceqtlgen/analysis_', dataset, '/')

In [14]:
dataset_name = 'CD4_T.Qced.Normalized.SCs.Rds'

# Functions

In [15]:
## Function to load dataset
get_dataset = function(path){
    data = readRDS(path)
    }

# Data

## Gene Pairing Data

In [16]:
## Load the gene-pairs (from correlation stats); to now for which pairs correlations have been calcualted

In [17]:
gene_pairs = read.csv(paste0(results_path_analysis, 'F6_Correlation_Summary_Stats_all.csv'))

In [19]:
gene_pairs = gene_pairs[,is.na(str_extract(colnames(gene_pairs), 'Unnamed'))]

In [20]:
gene_pairs = setDT(gene_pairs)

In [21]:
head(gene_pairs,2)

X,gene_pair,mean_correlation,mean_abs_correlation,var_correlation,max_correlation,n_NA,n_not_NA,n_significant,weighted_variance,perc_significant,weighted_var_div_mean
<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
0,A1BG-AS1_ARHGEF19,0.000254472,0.03102619,0.001349817,0.1113,918,100,3,0.0012404539,0.03000000,0.03998086
1,A1BG-AS1_B3GALNT2,-0.006959671,0.02835880,0.001029215,0.1173,828,190,6,0.0009689588,0.03157895,0.03416783


In [22]:
gene_pairs$gene1 = str_replace(gene_pairs$gene_pair, '_.*', '')
gene_pairs$gene2 = str_replace(gene_pairs$gene_pair, '.*_', '')

In [23]:
nrow(gene_pairs)

[1] 8649759

In [24]:
length(unique(gene_pairs$gene_pair))

[1] 8649759

## Load expression data

In [30]:
path = paste0(expression_data , dataset_name)
data = get_dataset(path)   

In [31]:
data

An object of class Seurat 
69334 features across 481032 samples within 2 assays 
Active assay: RNA (34667 features, 0 variable features)
 2 layers present: counts, data
 1 other assay present: data

In [32]:
#unique(data$Assignment)  - exemplary for one sample

# Calculate paired statistics

In [33]:
## Generate gene-pair chunks to iterate

In [34]:
choose_assay = 'RNA'

In [35]:
head(gene_pairs,2)

X,gene_pair,mean_correlation,mean_abs_correlation,var_correlation,max_correlation,n_NA,n_not_NA,n_significant,weighted_variance,perc_significant,weighted_var_div_mean,gene1,gene2
<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
0,A1BG-AS1_ARHGEF19,0.000254472,0.03102619,0.001349817,0.1113,918,100,3,0.0012404539,0.03000000,0.03998086,A1BG-AS1,ARHGEF19
1,A1BG-AS1_B3GALNT2,-0.006959671,0.02835880,0.001029215,0.1173,828,190,6,0.0009689588,0.03157895,0.03416783,A1BG-AS1,B3GALNT2


In [36]:
gene_pairs$group = rep(1:41, length.out = nrow(gene_pairs))

In [ ]:
### Empty dataframe to store results

In [37]:
paired_analysis_result = data.frame()

In [38]:
#gene_pairs %>% group_by(group) %>% dplyr::count()

In [39]:
unique(gene_pairs$group)

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41

In [40]:
expression_summary_result = data.frame()

In [ ]:
#for(i in unique(gene_pairs$group)){
for(i in 41){
    paired_analysis_result = data.frame()
    
    gene_pairs_subset = gene_pairs[gene_pairs$group == i,]
    
    for(j in unique(data$Assignment)){
        flush.console()
        print(j)
        sample_data = data[,data$Assignment == j]
        sample_data = GetAssayData(sample_data ,assay =choose_assay) 
        
        ### Filter genes with at leat 10 non-zero counts
        sample_data = sample_data[rowSums(sample_data >= 1) >= 10,]
        
        ### convert format
        sample_data = as.matrix(sample_data)
        data_long = melt(sample_data)
        colnames(data_long) = c('gene', 'cell', 'counts')
        
        ### Analyze paired values
        paired_analysis = merge(gene_pairs_subset[,c('gene1', 'gene2', 'gene_pair')], data_long, by.x = 'gene1', by.y = 'gene', allow.cartesian = TRUE)
        paired_analysis = merge(paired_analysis , data_long, by.x = c('gene2', 'cell'), by.y = c('gene', 'cell'))
        
        paired_analysis_summary = paired_analysis %>% group_by(gene_pair) %>% summarise(paired_non_zero = sum((counts.x != 0) & (counts.y != 0)),
                                                      paired_zero = sum((counts.x == 0) & (counts.y == 0)),
                                                      zero_one_cell = sum((counts.x == 0) | (counts.y == 0)),
                                                      non_zero_one_cell = sum((counts.x != 0) | (counts.y != 0))
                                                      )
        
        paired_analysis_summary$Assignment = j
        paired_analysis_result = rbind(paired_analysis_result, paired_analysis_summary)
        }
    print(i)
    
    expression_summary = paired_analysis_result %>% group_by(gene_pair) %>% summarise(n_sample = n(),   # aggregate across samples
                                                                                      mean_paired_non_zero = mean(paired_non_zero),
                                                                                      mean_paired_zero = mean(paired_zero),
                                                                                      mean_zero_one_cell = mean(zero_one_cell),
                                                                                      mean_non_zero_one_cell = mean(non_zero_one_cell))
    expression_summary$group = i
    write.csv(expression_summary, paste0(results_path_analysis, 'Paired_expression_summary', i, '.csv'))
    expression_summary_result  = rbind(expression_summary, expression_summary_result )
}    
    
    
        
        
        

[1] "692"
[1] "689"
[1] "690"
[1] "693"
[1] "688"
[1] "684"
[1] "687"
[1] "683"
[1] "685"
[1] "691"
[1] "694"
[1] "686"
[1] "826"
[1] "824"
[1] "833"
[1] "831"
[1] "828"
[1] "845"
[1] "829"
[1] "832"
[1] "844"
[1] "830"
[1] "827"
[1] "825"
[1] "843"
[1] "849"
[1] "835"
[1] "850"
[1] "986"
[1] "836"
[1] "846"
[1] "837"
[1] "848"
[1] "851"
[1] "998"
[1] "991"
[1] "852"
[1] "847"
[1] "990"
[1] "853"
[1] "1000"
[1] "996"
[1] "992"
[1] "999"
[1] "813"
[1] "811"
[1] "993"
[1] "987"
[1] "812"
[1] "995"
[1] "838"
[1] "997"
[1] "994"
[1] "1004"
[1] "1003"
[1] "859"
[1] "1001"
[1] "1002"
[1] "1005"
[1] "856"
[1] "858"
[1] "989"
[1] "854"
[1] "988"
[1] "857"
[1] "855"
[1] "860"
[1] "892"
[1] "862"
[1] "1007"
[1] "805"
[1] "891"
[1] "863"
[1] "804"
[1] "1006"
[1] "865"
[1] "1008"
[1] "861"
[1] "864"
[1] "803"
[1] "1010"
[1] "1011"
[1] "806"
[1] "808"
[1] "894"
[1] "1012"
[1] "1014"
[1] "1009"
[1] "807"
[1] "1015"
[1] "893"
[1] "809"
[1] "810"
[1] "1013"
[1] "1022"
[1] "1020"
[1] "897"
[1] "1016"
[

In [43]:
head(paired_analysis_result,5)

gene_pair,paired_non_zero,paired_zero,zero_one_cell,non_zero_one_cell,Assignment
<chr>,<int>,<int>,<int>,<int>,<chr>
A1BG-AS1_ANKS6,0,666,709,43,692
A1BG-AS1_ARHGAP18,1,659,708,50,692
A1BG-AS1_CDK2,0,669,709,40,692
A1BG-AS1_DMXL2,0,663,709,46,692
A1BG-AS1_DNAH11,0,667,709,42,692
